# Taking over the event loop
* https://ipython.readthedocs.io/en/stable/config/eventloops.html#integrating-with-a-new-event-loop-in-the-kernel

# Status
This hasn't worked as I hoped. My implementation in `trio` of a kernel event loop, to allow other work to proceed as well, works maybe one of five runs. It's obvious I don't understand how to do this.

In [ ]:
from ipykernel.eventloops import register_integration
import ipywidgets as widgets
from IPython.display import display
import math
import time

In [ ]:
%gui fui

# Circulation sliders

In [ ]:
from math import pi

v_w = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Sin:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='.2f',
)

h_w = widgets.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.01,
    description='Cos:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

t_w = widgets.IntSlider(
    description='t',
)

shutdown_b_w = widgets.Button(description="Shutdown worker")
clear_b_w = widgets.Button(description="clear")
out_w = widgets.Output(layout={'border': '1px solid black'})
display(v_w, h_w, t_w, shutdown_b_w, clear_b_w, out_w)

# Trio version of event loop

In [ ]:
async def circulate(t, v, h):
    while True:
        #print(f"t slider at {t.value}")
        t_f = (t.value / 100) * 2 * math.pi
        v.value = math.sin(t_f)
        h.value = math.cos(t_f)
        await trio.sleep(0.1)

In [ ]:
import trio

observations = dict()
def take_note_of(value, name):
    observations[name] = value
    
@register_integration('tri_2')
def loop_tri_2(kernel):
    async def tri_loop_2():
        #take_note_of(kernel._poll_interval, 'kpi')
        #take_note_of(kernel.do_one_iteration, 'doi')
        while True:
            kernel.do_one_iteration()
            await trio.sleep(10 * kernel._poll_interval)
            #await trio.sleep(0)

    async def both():
        async with trio.open_nursery() as nursery:
            nursery.start_soon(tri_loop_2)
            #await trio.sleep(0.5)
            nursery.start_soon(circulate, t_w, v_w, h_w)

    trio.run(both)

In [ ]:
time.sleep(1)

In [ ]:
%gui tri_2

In [ ]:
time.sleep(1)

In [ ]:
1+1

In [ ]:
assert False, "stop here"

In [ ]:
%gui

---

# The idiot's event loop

In [ ]:
@register_integration('foo')
def loop_foo(kernel):
    """Start a kernel with an idiot's event loop."""
    import time

    while True:
        kernel.do_one_iteration()
        time.sleep(kernel._poll_interval)

In [ ]:
%gui foo

In [ ]:
1+1+1

## Two sliders linked in the kernel

In [ ]:
a = widgets.FloatSlider()
b = widgets.FloatSlider()
l = widgets.link((a, 'value'), (b, 'value'))
display(a,b)

#mylink = widgets.jslink((a, 'value'), (b, 'value'))

In [ ]:
mylink.unlink()

In [ ]:
%gui

# with more latency

In [ ]:
@register_integration('slofoo')
def loop_slofoo(kernel):
    """Start a kernel with an idiot's event loop, but slower."""
    import time

    while True:
        kernel.do_one_iteration()
        time.sleep(100 * kernel._poll_interval)

In [ ]:
%gui slofoo

In [ ]:
1+1+1

# Trio?

In [ ]:
import trio

@register_integration('tri_1')
def loop_tri_1(kernel):
    async def tri_loop_1():
        while True:
            kernel.do_one_iteration()
            await trio.sleep(kernel._poll_interval)

    trio.run(tri_loop_1)

In [ ]:
%gui tri_1

In [ ]:
1+1

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
%gui 

In [ ]:
1+1

In [ ]:
%gui

In [ ]:
observed_kpi, observed_doi